<a href="https://colab.research.google.com/github/Hari0077/FYP_TNTD/blob/main/VGG16_BM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras import metrics
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/VGG_BM '

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/VGG_BM ')

In [ ]:
# Flatten the output layer.
x = Flatten()(vgg.output)

In [ ]:
# Dense layer is added.
prediction = Dense(1, activation='sigmoid')(x)


# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/VGG_BM ',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 334 images belonging to 2 classes.


In [ ]:
import keras

class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
            self.model.save("model_{}.hd5".format(epoch))

In [ ]:
saver = CustomSaver()

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  epochs=3,
  callbacks=[saver],
  steps_per_epoch=len(training_set),
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/3
11/11 [==============================] - 176s 15s/step - loss: 0.8467 - accuracy: 0.5509
Epoch 2/3
11/11 [==============================] - 171s 15s/step - loss: 0.6977 - accuracy: 0.5778
Epoch 3/3
11/11 [==============================] - 176s 16s/step - loss: 0.6402 - accuracy: 0.6317


In [ ]:
# save it as a h5 file

import tensorflow as tf


from keras.models import load_model

model.save('model_BM_vgg16.h5')

In [ ]:
from keras.models import load_model

model = load_model("/content/model_BM_vgg16.h5")
accuracy=model.evaluate_generator(training_set,steps=32)
print("Accuracy",accuracy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  after removing the cwd from sys.path.


Accuracy [0.6174076795578003, 0.667664647102356]


In [ ]:
!pip install gTTS
!pip install google_trans_new
#!pip install googletrans==3.1.0a0
!pip install translate

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import load_model
image_path = '/content/drive/MyDrive/VGG_BM /Malign/105_1.jpg'
image = load_img(image_path, target_size=(224, 224))
      # convert the image pixels to a numpy array
image = img_to_array(image)
      # reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
      # prepare the image for the VGG model
image = preprocess_input(image)
      # predict the probability across all output classes
model_output = model.predict(image)

#take maximum value from the model

#print(model_output)
# print('The tirad type is ',predcls)

if model_output <=0.5:
  print("The tirad type is Benign")
else:
  print("The tirad type is Malign")


The tirad type is Malign
